# Importing essential functions

In [1]:
! git clone https://github.com/Priyansi/pytorch-chatbot.git

Cloning into 'pytorch-chatbot'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


# Train

In [2]:
PATH = '/content/pytorch-chatbot/'

In [5]:
import sys
sys.path.append(PATH)

In [6]:
import json
import numpy as np
from nltk_utils import *
from torch.utils.data import (Dataset, DataLoader)

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
with open(f'{PATH}intents.json', 'r') as file:
    intents = json.load(file)

In [13]:
IGNORE_WORDS = ['?', '.', '!', ',', ':', ';']

In [14]:
all_words = []
tags = []
xy = []
x_train = []
y_train = []

In [15]:
for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
      words = tokenize(pattern)
      words = [stem(word) for word in words if word not in IGNORE_WORDS]
      all_words.extend(words)
      xy.append((words, tag))
all_words = sorted(set(all_words))
tags = sorted(set(tags))

for (stemmed_sentence, tag) in xy:
  bog = bag_of_words(stemmed_sentence, all_words)
  x_train.append(bog)

  label = tags.index(tag)
  y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [16]:
class ChatDataset(Dataset):
    def __init__(self, x_train, y_train):
        self.num_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    def __getitem__(self, ind):
        return self.x_data[ind], self.y_data[ind]

    def __len__(self):
        return self.num_samples

In [17]:
batch_size = 8
learning_rate = 0.001
num_epochs = 600

In [18]:
dataset = ChatDataset(x_train, y_train)
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

# Model

In [19]:
import torch
import torch.nn as nn

In [20]:
input_size = len(x_train[0])
hidden_size = 16
num_classes = len(tags)

In [21]:
class FeedForwardNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(FeedForwardNet, self).__init__()
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, hidden_size)
    self.layer3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.layer1(x)
    out = self.relu(out)
    out = self.layer2(out)
    out = self.relu(out)
    out = self.layer3(out)
    return out

In [22]:
model = FeedForwardNet(input_size, hidden_size, num_classes)
model

FeedForwardNet(
  (layer1): Linear(in_features=62, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=16, bias=True)
  (layer3): Linear(in_features=16, out_features=10, bias=True)
  (relu): ReLU()
)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [24]:
loss_func = nn.CrossEntropyLoss()
optimizer_func = torch.optim.Adam

In [25]:
def train(num_epochs, learning_rate, train_loader, loss_func, optimizer_func=torch.optim.SGD):
  optimizer = optimizer_func(model.parameters(), lr = learning_rate)
  for epoch in range(num_epochs):
    for (words, labels) in train_loader:
      words = words.to(device)
      labels = labels.to(device)

      outputs = model(words)
      loss = loss_func(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if (epoch+1)%100 == 0 :
      print("Epoch [{}] : Loss : {}".format(epoch, loss.item()))

In [26]:
train(num_epochs, learning_rate, train_loader, loss_func, optimizer_func)

Epoch [99] : Loss : 0.2734623849391937
Epoch [199] : Loss : 0.012202807702124119
Epoch [299] : Loss : 0.012430126778781414
Epoch [399] : Loss : 0.005026794504374266
Epoch [499] : Loss : 0.0016176613280549645
Epoch [599] : Loss : 0.0006242948002181947


In [28]:
data = {
    'model_state' : model.state_dict(),
    'input_size' : input_size,
    'hidden_size' : hidden_size,
    'output_size' : num_classes,
    'all_words' : all_words,
    'tags' : tags
}

In [31]:
FILE = 'data.pth'
torch.save(data, FILE)